## Project Scenario

You have been hired as a data engineer by research organization. Your boss has asked you to create a code that can be used to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, the data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information that has been made available to you as a CSV file. The processed information table is to be saved locally in a CSV format and as a database table.

Your job is to create an automated system to generate this information so that the same can be executed in every financial quarter to prepare the report.

Particulars of the code to be made have been shared below.

![image](info.jpg)

## Directions

1. Write a function to extract the tabular information from the given URL under the heading By Market Capitalization, and save it to a data frame.
2. Write a function to transform the data frame by adding columns for Market Capitalization in GBP, EUR, and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.
3. Write a function to load the transformed data frame to an output CSV file.
4. Write a function to load the transformed data frame to an SQL database server as a table.
5. Write a function to run queries on the database table.
6. Run the following queries on the database table:
    <br>a. Extract the information for the London office, that is Name and MC_GBP_Billion
    <br>b. Extract the information for the Berlin office, that is Name and MC_EUR_Billion
    <br>c. Extract the information for New Delhi office, that is Name and MC_INR_Billion
7. Write a function to log the progress of the code.
8. While executing the data initialization commands and function calls, maintain appropriate log entries.

## Project tasks

Task 1:
Write a function log_progress() to log the progress of the code at different stages in a file code_log.txt. Use the list of log points provided to create log entries as every stage of the code.

Task 2:
Extract the tabular information from the given URL under the heading 'By market capitalization' and save it to a dataframe.
<br>a. Inspect the webpage and identify the position and pattern of the tabular information in the HTML code
<br>b. Write the code for a function extract() to perform the required data extraction.
<br>c. Execute a function call to extract() to verify the output.

Task 3:
Transform the dataframe by adding columns for Market Capitalization in GBP, EUR and INR, rounded to 2 decimal places, based on the exchange rate information shared as <br>a CSV file.
<br>a. Write the code for a function transform() to perform the said task.
<br>b. Execute a function call to transform() and verify the output.

Task 4:
Load the transformed dataframe to an output CSV file. Write a function load_to_csv(), execute a function call and verify the output.

Task 5:
Load the transformed dataframe to an SQL database server as a table. Write a function load_to_db(), execute a function call and verify the output.

Task 6:
Run queries on the database table. Write a function load_from_db(), execute a given set of queries and verify the output.

Task 7:
Verify that the log entries have been completed at all stages by checking the contents of the file code_log.txt.

## Let's start!

In [ ]:
from datetime import  datetime
import pandas as pd
import sqlite3
import requests
from bs4 import BeautifulSoup

Define variables

In [ ]:
logfile="code_log.txt"
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
csv_file = "./Largest_banks_data.csv"
db_name = "Banks.db"
table_name = "Largest_banks"

Task 1:
Write a function log_progress() to log the progress of the code at different stages in a file code_log.txt. Use the list of log points provided to create log entries as every stage of the code.

In [ ]:
def log_progress(message, logfile=logfile):
    now = datetime.now()
    timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
    with open(logfile, "a") as f:
        f.write(timestamp + ", " + message + "\n")

Task 2:
Extract the tabular information from the given URL under the heading 'By market capitalization' and save it to a dataframe.
<br>a. Inspect the webpage and identify the position and pattern of the tabular information in the HTML code
<br>b. Write the code for a function extract() to perform the required data extraction.
<br>c. Execute a function call to extract() to verify the output.

In [ ]:
def extract(url):
    df = pd.read_html(url)[0]
    df = df.rename({"Bank name": "Name", "Market cap (US$ billion)": "MC_USD_Billion"}, axis=1)
    df = df.drop("Rank", axis=1)
    log_progress(message="Data has been extracted")
    return df


def extract_v2(url):
    html_page = requests.get(url=url).text
    data = BeautifulSoup(html_page, "html.parser")
    tables = data.find_all("tbody")
    rows = tables[0].find_all("tr")
    df = pd.DataFrame(columns=["Name", "MC_USD_Billion"])
    for row in rows:
        col = row.find_all("td")
        if len(col) != 0:
            data_dict = {"Name": col[1].find_all("a")[1],
                        "MC_USD_Billion": col[2].text.split("\n")[0]}
            # data_dict = {"Name": col[1].find_all("a")[1],
            # "MC_USD_Billion": float(col[2].contents[0][:-1])}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1], ignore_index=True)
    log_progress(message="Data has been extracted")
    return df


market_caps_table = extract_v2(url=url)
market_caps_table

In [ ]:
market_caps_table.info()

Task 3:
Transform the dataframe by adding columns for Market Capitalization in GBP, EUR and INR, rounded to 2 decimal places, based on the exchange rate information shared as <br>a CSV file.
<br>a. Write the code for a function transform() to perform the said task.
<br>b. Execute a function call to transform() and verify the output.

In [ ]:
def transform(df):
    exchange_rate = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv")
    df["MC_USD_Billion"] = df["MC_USD_Billion"].transform(lambda x: float(x))
    df["MC_GBP_Billion"] = df["MC_USD_Billion"].transform(lambda x: round(x*exchange_rate.loc[1, "Rate"], 2))
    df["MC_EUR_Billion"] = df["MC_USD_Billion"].transform(lambda x: round(x*exchange_rate.loc[0, "Rate"], 2))
    df["MC_INR_Billion"] = df["MC_USD_Billion"].transform(lambda x: round(x*exchange_rate.loc[2, "Rate"], 2))
    log_progress(message="Data has been transformed")
    return df

market_caps_table = transform(df=market_caps_table)
market_caps_table

Task 4:
Load the transformed dataframe to an output CSV file. Write a function load_to_csv(), execute a function call and verify the output.

In [ ]:
def load_to_csv(df, csv_file):
    df.to_csv(csv_file)
    log_progress(message="Data has been loaded to CSV")

load_to_csv(df=market_caps_table, csv_file=csv_file)

Task 5:
Load the transformed dataframe to an SQL database server as a table. Write a function load_to_db(), execute a function call and verify the output.

In [ ]:
def load_to_db(df, db_name, table_name):
    conn = sqlite3.connect(db_name)
    df.to_sql(name=table_name, con=conn, if_exists='replace', index=False)
    conn.close()
    log_progress(message="Data has been loaded to DB")

load_to_db(df=market_caps_table, db_name=db_name, table_name=table_name)

Task 6:
Run queries on the database table. Write a function load_from_db(), execute a given set of queries and verify the output.

In [ ]:
def load_from_db(df, db_name, query):
    conn = sqlite3.connect(db_name)
    db_table = pd.read_sql(sql=query, con=conn)
    conn.close()
    log_progress(message="Data on DB has been checked")
    return db_table

load_from_db(df=market_caps_table, db_name=db_name, query=f"SELECT * from {table_name}")

In [ ]:
load_from_db(df=market_caps_table, db_name=db_name, query=f"SELECT AVG(MC_GBP_Billion) from {table_name}")

Task 7:
Verify that the log entries have been completed at all stages by checking the contents of the file code_log.txt.

In [ ]:
with open(logfile, "r") as f:
    all_logs = f.readlines()
    for log in all_logs:
        print(log)